In [1]:
# Gerekli kütüphaneler
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hiperparametreler
num_epochs = 5
batch_size = 100
learning_rate = 0.1

In [5]:
# MNIST Veri Kümesini Yükleme ve Normalizasyon
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)



In [7]:
# CNN Modeli
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected layer
        self.fc1 = nn.Linear(32 * 4 * 4, 10) 
    
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        
        # Flatten
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        
        return out

# Model, Loss ve Optimizer Tanımlama
model = CNNModel().to(device)
error = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
# Model Eğitimi
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Reshaping (28x28 -> 1x28x28)
        train = images.view(-1, 1, 28, 28)
        
        # Gradients sıfırlama
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(train)
        
        # Loss hesaplama
        loss = error(outputs, labels)
        
        # Backward propagation
        loss.backward()
        
        # Parametre güncelleme
        optimizer.step()
        
        count += 1
        
        if count % 50 == 0:
            # Test Seti Üzerinde Modeli Değerlendir
            correct = 0
            total = 0
            with torch.no_grad():
                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
                    test = images.view(-1, 1, 28, 28)
                    outputs = model(test)
                    
                    # En yüksek olasılığı seç
                    predicted = torch.max(outputs.data, 1)[1]
                    
                    # Doğru tahminleri say
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            
            accuracy = 100 * correct / total
            
            # Kayıtları tut
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            
        if count % 200 == 0:
            print(f'Iteration: {count}  Loss: {loss.item()}  Accuracy: {accuracy:.2f}%')

print("Eğitim Tamamlandı!")

Iteration: 200  Loss: 0.0021394784562289715  Accuracy: 98.83%
Iteration: 400  Loss: 0.019065890461206436  Accuracy: 98.71%
Iteration: 600  Loss: 0.04052665829658508  Accuracy: 98.89%
Iteration: 800  Loss: 0.01295552495867014  Accuracy: 98.90%
Iteration: 1000  Loss: 0.052454303950071335  Accuracy: 99.04%
Iteration: 1200  Loss: 0.026968233287334442  Accuracy: 98.94%
Iteration: 1400  Loss: 0.05838070437312126  Accuracy: 98.88%
Iteration: 1600  Loss: 0.029644260182976723  Accuracy: 98.86%
Iteration: 1800  Loss: 0.0034467303194105625  Accuracy: 99.01%
Iteration: 2000  Loss: 0.04986675828695297  Accuracy: 98.98%
Iteration: 2200  Loss: 0.0031733077485114336  Accuracy: 98.92%
Iteration: 2400  Loss: 0.021058611571788788  Accuracy: 98.86%
Iteration: 2600  Loss: 0.0016769240610301495  Accuracy: 98.99%
Iteration: 2800  Loss: 0.03108522854745388  Accuracy: 98.99%
Iteration: 3000  Loss: 0.02099771238863468  Accuracy: 98.99%
Eğitim Tamamlandı!
